In [1]:
import pandas as pd
import numpy as np

ratings_df = pd.read_csv('datasets/ml-latest-small/ratings.csv')
movies = pd.read_csv('datasets/ml-latest-small/movies.csv', index_col=0)

ratings_df.columns = ['user_id', 'movie_id', 'rating', 'timestamp']

user_ratings = ratings_df.pivot(index='movie_id', columns='user_id', values='rating')

In [65]:
user = user_ratings.sample(axis=1).assign(title=movies.title[user_ratings.index])
user.columns = ['rating', 'title']
user.sort_values(by='rating', ascending=False).head(20)

,rating,title
movie_id,,
1258,5.0,"Shining, The (1980)"
1255,5.0,Bad Taste (1987)
1349,5.0,Vampire in Venice (Nosferatu a Venezia) (Nosfe...
1348,5.0,"Nosferatu (Nosferatu, eine Symphonie des Graue..."
1339,5.0,Dracula (Bram Stoker's Dracula) (1992)
1387,5.0,Jaws (1975)
247,5.0,Heavenly Creatures (1994)
1333,4.0,"Birds, The (1963)"
1261,4.0,Evil Dead II (Dead by Dawn) (1987)


In [66]:
from sklearn.neighbors import NearestNeighbors
ratings_mask = np.isfinite(user_ratings.values).T
user_matrix = np.nan_to_num(user_ratings.values).T

nnbrs = NearestNeighbors(n_neighbors=5, algorithm='brute', metric='cosine').fit(user_matrix)


In [67]:
distance, neighbors = nnbrs.kneighbors(np.nan_to_num(user.rating.values.reshape(1, -1)))

similarity = 1 - distance.flatten()[1:]

neighbor_mask = ratings_mask[neighbors.flatten()[1:]]
neighbors = user_matrix[neighbors.flatten()[1:]]

np.seterr(divide='ignore', invalid='ignore')

pred = similarity @ neighbors
pred = pred / (similarity * neighbor_mask.T).sum(axis=1)

pred = np.nan_to_num(pred).flatten()
pred.shape

(9066,)

In [69]:
user['predicted'] = pred
user.sort_values(by='predicted', ascending=False).head(20)

,rating,title,predicted
movie_id,,,
2650,NaN,"Ghost of Frankenstein, The (1942)",5.0
2652,NaN,"Curse of Frankenstein, The (1957)",5.0
2634,NaN,"Mummy, The (1959)",5.0
2644,NaN,Dracula (1931),5.0
1721,NaN,Titanic (1997),5.0
2647,NaN,House of Frankenstein (1944),5.0
2648,NaN,Frankenstein (1931),5.0
2649,NaN,Son of Frankenstein (1939),5.0
2636,NaN,"Mummy's Ghost, The (1944)",5.0
